In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime

### Прогноз только на основании ИПЦ

In [3]:
fileName = 'ipc_mes_02-2024.xlsx'
excelColumns = 'B:AI'
sheetName = '01'
df = pd.read_excel(
            fileName,
            sheet_name=sheetName,
            usecols=excelColumns,
            nrows=12,
            skiprows=[0, 1, 2, 4],
            header=0
        )

In [6]:
transposed_df = df.transpose()
cpi_series = transposed_df.stack()
new_index = cpi_series.index.map(lambda x: datetime.strptime(f'{x[0]}-{x[1]+1}', '%Y-%m'))
cpi_series.index = new_index
cpi_series.index.freq = 'MS'

In [121]:
cpi_series

1991-01-01    106.20
1991-02-01    104.80
1991-03-01    106.30
1991-04-01    163.50
1991-05-01    103.00
               ...  
2023-10-01    100.83
2023-11-01    101.11
2023-12-01    100.73
2024-01-01    100.86
2024-02-01    100.68
Freq: MS, Length: 398, dtype: float64

In [122]:
train, test = train_test_split(cpi_series, test_size=0.2, shuffle=False)

In [123]:
order = (1, 0, 1)  # Example SARIMA order
seasonal_order = (1, 1, 1, 12)  # Example seasonal order
model = sm.tsa.SARIMAX(train, order=order, seasonal_order=seasonal_order)
fit_model = model.fit(disp=False)

In [124]:
predictions = fit_model.predict(
    start=test.index[0], 
    end=test.index[-1],
    exog=test,
    dynamic=True
)
mae = mean_absolute_error(test, predictions)
rmse = np.sqrt(mean_squared_error(test, predictions))
print(mae, rmse)

0.41350087578109596 0.9028664227729162


### Добавим ключевую ставку

In [72]:
fileName = 'key_index.xlsx'
sheetName = '3'
df_key_rate = pd.read_excel(
            fileName,
            sheet_name=sheetName,
            #usecols=excelColumns,
            #nrows=12,
            #skiprows=[0, 1, 2, 4],
            header=0
        )

In [80]:
df_key_rate = df_key_rate.sort_values(by='Дата').reset_index(drop=True)

In [84]:
df_new = pd.DataFrame()
for date in pd.date_range(start=df_key_rate['Дата'].iloc[0], end=df_key_rate['Дата'].iloc[-1], freq='MS'):
    value = df_key_rate.loc[(df_key_rate['Дата'] - date).abs().idxmin(), 'Ставка']
    df_new = df_new.append({'Дата': date, 'Ставка': value}, ignore_index=True)

df_key_rate = df_new

In [85]:
df_key_rate

,Дата,Ставка
0,2013-10-01,5.5
1,2013-11-01,5.5
2,2013-12-01,5.5
3,2014-01-01,5.5
4,2014-02-01,5.5
...,...,...
122,2023-12-01,15.0
123,2024-01-01,16.0
124,2024-02-01,16.0
125,2024-03-01,16.0


#### До 2013 ставка рефинансирования

In [74]:
fileName = 'key_index.xlsx'
sheetName = '2'
df_key_rate2 = pd.read_excel(
            fileName,
            sheet_name=sheetName,
            #usecols=excelColumns,
            #nrows=12,
            #skiprows=[0, 1, 2, 4],
            header=0
        )

In [75]:
df_key_rate2

,Начало,Конец,%
0,14 сентября 2012 г.,31 декабря 2015 г.,8.25
1,26 декабря 2011 г.,13 сентября 2012 г.,8.00
2,3 мая 2011 г.,25 декабря 2011 г.,8.25
3,28 февраля 2011 г.,2 мая 2011 г.,8.00
4,1 июня 2010 г.,27 февраля 2011 г.,7.75
...,...,...,...
79,2 июня 1993 г.,21 июня 1993 г.,110.00
80,30 марта 1993 г.,1 июня 1993 г.,100.00
81,23 мая 1992 г.,29 марта 1993 г.,80.00
82,10 апреля 1992 г.,22 мая 1992 г.,50.00


In [76]:
dictianory = {
    'января': '01', 
    'февраля': '02', 
    'марта': '03', 
    'апреля': '04', 
    'мая': '05', 
    'июня': '06', 
    'июля': '07', 
    'августа': '08', 
    'сентября': '09', 
    'октября': '10', 
    'ноября': '11', 
    'декабря': '12', 
}

def convert_date(date_str):
    date_str = date_str.strip()
    date_str = date_str.replace('\xa0', '')
    for key in dictianory:
        if key in date_str:
            date_str = date_str.replace(key, '.' + dictianory[key] + '.')
    date_str = date_str.replace('г.', '')
    date_str = date_str.replace(' ', '')
    date = datetime.strptime(date_str, '%d.%m.%Y')
    return date #date.strftime('%d.%m.%Y')

df_key_rate2['Начало'] = df_key_rate2['Начало'].apply(convert_date)
df_key_rate2['Конец'] = df_key_rate2['Конец'].apply(convert_date)

In [77]:
df_key_rate2

,Начало,Конец,%
0,2012-09-14,2015-12-31,8.25
1,2011-12-26,2012-09-13,8.00
2,2011-05-03,2011-12-25,8.25
3,2011-02-28,2011-05-02,8.00
4,2010-06-01,2011-02-27,7.75
...,...,...,...
79,1993-06-02,1993-06-21,110.00
80,1993-03-30,1993-06-01,100.00
81,1992-05-23,1993-03-29,80.00
82,1992-04-10,1992-05-22,50.00


In [134]:
final_df = pd.DataFrame(cpi_series).reset_index()

In [135]:
final_df = pd.merge(final_df, df_key_rate, how='left', left_on='index', right_on='Дата')

In [136]:
final_df = final_df.drop(columns=['Дата'])

In [137]:
final_df = final_df.rename(columns={"index": "Дата", 0: "ИПЦ"})

In [138]:
final_df['%'] = final_df['Дата'].apply(
    lambda x: df2.loc[(x >= df2['Начало']) & (x <= df2['Конец']), '%'].values[0] 
    if not df2.loc[(x >= df2['Начало']) & (x <= df2['Конец'])].empty else None
)

In [139]:
mask = final_df['Дата'] < pd.to_datetime('2013-10-01')
final_df.loc[mask,'Ставка'] = final_df.loc[mask,'%'] 

In [140]:
final_df = final_df[final_df['Ставка'].notna()]

In [141]:
final_df.set_index('Дата', inplace=True)
final_df.index.freq = 'MS'

### Обучаем модель

#### С учетом обрезки 1991 года без ключевой ставки

In [142]:
train, test = train_test_split(final_df, test_size=0.2, shuffle=False)

In [143]:
order = (1, 0, 1)
seasonal_order = (1, 1, 1, 12)
model = sm.tsa.SARIMAX(
    train['ИПЦ'], 
    #exog=train['Ставка'],
    order=order, 
    seasonal_order=seasonal_order)
fit_model = model.fit(disp=False)

In [144]:
predictions = fit_model.predict(
    start=test.index[0], 
    end=test.index[-1],
    #exog=test['Ставка'],
    dynamic=True
)
mae = mean_absolute_error(test['ИПЦ'], predictions)
rmse = np.sqrt(mean_squared_error(test['ИПЦ'], predictions))
print(mae, rmse)

1.0093805165245135 1.3902868877362764


#### С ключевой ставкой

In [146]:
order = (1, 0, 1)
seasonal_order = (1, 1, 1, 12)
model = sm.tsa.SARIMAX(
    train['ИПЦ'], 
    exog=train['Ставка'],
    order=order, 
    seasonal_order=seasonal_order)
fit_model = model.fit(disp=False)

In [147]:
predictions = fit_model.predict(
    start=test.index[0], 
    end=test.index[-1],
    exog=test['Ставка'],
    dynamic=True
)
mae = mean_absolute_error(test['ИПЦ'], predictions)
rmse = np.sqrt(mean_squared_error(test['ИПЦ'], predictions))
print(mae, rmse)

0.977911748917865 1.3728803949922144
